# Store Sales Prediction API Testing

This notebook contains comprehensive tests for the Store Sales Prediction FastAPI server.
It tests all endpoints including health checks, single predictions, batch predictions, and CSV uploads.

In [1]:
# Import required libraries for testing
import requests
import json
import pandas as pd
import time
from datetime import datetime, timedelta
import io
from typing import Dict, List

# Base URL for the API (make sure your server is running)
BASE_URL = "http://localhost:8000"

print("Testing libraries imported successfully!")
print(f"Testing API at: {BASE_URL}")

Testing libraries imported successfully!
Testing API at: http://localhost:8000


In [2]:
# Helper function to check if server is running
def check_server_status():
    """Check if the FastAPI server is running"""
    try:
        response = requests.get(f"{BASE_URL}/health", timeout=5)
        if response.status_code == 200:
            print("✅ Server is running!")
            print(f"Health check response: {response.json()}")
            return True
        else:
            print(f"❌ Server responded with status code: {response.status_code}")
            return False
    except requests.exceptions.ConnectionError:
        print("❌ Cannot connect to server. Make sure it's running on port 8000")
        return False
    except requests.exceptions.Timeout:
        print("❌ Server request timed out")
        return False

# Check server status
server_running = check_server_status()

✅ Server is running!
Health check response: {'status': 'healthy', 'pipeline_loaded': True, 'reference_data_loaded': True, 'timestamp': '2025-06-01T22:53:47.689751'}


In [3]:
# Test 1: Root endpoint
def test_root_endpoint():
    """Test the root endpoint"""
    print("\n🧪 Testing Root Endpoint (/)")
    try:
        response = requests.get(f"{BASE_URL}/")
        print(f"Status Code: {response.status_code}")
        print(f"Response: {response.json()}")
        
        if response.status_code == 200:
            print("✅ Root endpoint test passed!")
        else:
            print("❌ Root endpoint test failed!")
    except Exception as e:
        print(f"❌ Error testing root endpoint: {e}")

if server_running:
    test_root_endpoint()


🧪 Testing Root Endpoint (/)
Status Code: 200
Response: {'message': 'Store Sales Prediction API', 'status': 'healthy'}
✅ Root endpoint test passed!
Status Code: 200
Response: {'message': 'Store Sales Prediction API', 'status': 'healthy'}
✅ Root endpoint test passed!


In [4]:
# Test 2: Health check endpoint
def test_health_endpoint():
    """Test the health check endpoint"""
    print("\n🧪 Testing Health Check Endpoint (/health)")
    try:
        response = requests.get(f"{BASE_URL}/health")
        print(f"Status Code: {response.status_code}")
        health_data = response.json()
        print(f"Response: {health_data}")
        
        # Check if all components are loaded
        if health_data.get('pipeline_loaded') and health_data.get('reference_data_loaded'):
            print("✅ All components loaded successfully!")
        else:
            print("⚠️ Some components may not be loaded properly")
            
        if response.status_code == 200:
            print("✅ Health check test passed!")
        else:
            print("❌ Health check test failed!")
    except Exception as e:
        print(f"❌ Error testing health endpoint: {e}")

if server_running:
    test_health_endpoint()


🧪 Testing Health Check Endpoint (/health)
Status Code: 200
Response: {'status': 'healthy', 'pipeline_loaded': True, 'reference_data_loaded': True, 'timestamp': '2025-06-01T22:54:05.113042'}
✅ All components loaded successfully!
✅ Health check test passed!
Status Code: 200
Response: {'status': 'healthy', 'pipeline_loaded': True, 'reference_data_loaded': True, 'timestamp': '2025-06-01T22:54:05.113042'}
✅ All components loaded successfully!
✅ Health check test passed!


In [5]:
# Test 3: Model info endpoint
def test_model_info():
    """Test the model info endpoint"""
    print("\n🧪 Testing Model Info Endpoint (/model/info)")
    try:
        response = requests.get(f"{BASE_URL}/model/info")
        print(f"Status Code: {response.status_code}")
        print(f"Response: {response.json()}")
        
        if response.status_code == 200:
            print("✅ Model info test passed!")
        else:
            print("❌ Model info test failed!")
    except Exception as e:
        print(f"❌ Error testing model info endpoint: {e}")

if server_running:
    test_model_info()


🧪 Testing Model Info Endpoint (/model/info)
Status Code: 200
Response: {'model_type': "<class 'sklearn.pipeline.Pipeline'>", 'status': 'loaded', 'pipeline_steps': ['feature_engineering', 'preprocessing', 'model_selector'], 'feature_engineering_steps': ['temporal_features', 'holiday_features', 'external_data', 'lag_features', 'rolling_features', 'null_handler']}
✅ Model info test passed!
Status Code: 200
Response: {'model_type': "<class 'sklearn.pipeline.Pipeline'>", 'status': 'loaded', 'pipeline_steps': ['feature_engineering', 'preprocessing', 'model_selector'], 'feature_engineering_steps': ['temporal_features', 'holiday_features', 'external_data', 'lag_features', 'rolling_features', 'null_handler']}
✅ Model info test passed!


In [6]:
# Test 4: Single prediction
def test_single_prediction():
    """Test single prediction endpoint"""
    print("\n🧪 Testing Single Prediction Endpoint (/predict)")
    
    # Sample prediction request
    prediction_data = {
        "id": 1,
        "date": "2023-08-01",
        "store_nbr": 1,
        "family": "GROCERY I",
        "onpromotion": 5
    }
    
    try:
        response = requests.post(
            f"{BASE_URL}/predict",
            json=prediction_data,
            headers={"Content-Type": "application/json"}
        )
        
        print(f"Request data: {prediction_data}")
        print(f"Status Code: {response.status_code}")
        print(f"Response: {response.json()}")
        
        if response.status_code == 200:
            result = response.json()
            if 'predicted_sales' in result:
                print(f"✅ Predicted sales: {result['predicted_sales']}")
                print("✅ Single prediction test passed!")
            else:
                print("❌ Response missing predicted_sales field")
        else:
            print("❌ Single prediction test failed!")
            print(f"Error details: {response.text}")
    except Exception as e:
        print(f"❌ Error testing single prediction: {e}")

if server_running:
    test_single_prediction()


🧪 Testing Single Prediction Endpoint (/predict)
Request data: {'id': 1, 'date': '2023-08-01', 'store_nbr': 1, 'family': 'GROCERY I', 'onpromotion': 5}
Status Code: 200
Response: {'id': 1, 'predicted_sales': 11.8190298783977, 'status': 'success'}
✅ Predicted sales: 11.8190298783977
✅ Single prediction test passed!
Request data: {'id': 1, 'date': '2023-08-01', 'store_nbr': 1, 'family': 'GROCERY I', 'onpromotion': 5}
Status Code: 200
Response: {'id': 1, 'predicted_sales': 11.8190298783977, 'status': 'success'}
✅ Predicted sales: 11.8190298783977
✅ Single prediction test passed!


In [7]:
# Test 5: Batch prediction
def test_batch_prediction():
    """Test batch prediction endpoint"""
    print("\n🧪 Testing Batch Prediction Endpoint (/predict/batch)")
    
    # Sample batch prediction request
    batch_data = {
        "predictions": [
            {
                "id": 1,
                "date": "2023-08-01",
                "store_nbr": 1,
                "family": "GROCERY I",
                "onpromotion": 5
            },
            {
                "id": 2,
                "date": "2023-08-01",
                "store_nbr": 2,
                "family": "BEVERAGES",
                "onpromotion": 3
            },
            {
                "id": 3,
                "date": "2023-08-02",
                "store_nbr": 1,
                "family": "DAIRY",
                "onpromotion": 0
            }
        ]
    }
    
    try:
        response = requests.post(
            f"{BASE_URL}/predict/batch",
            json=batch_data,
            headers={"Content-Type": "application/json"}
        )
        
        print(f"Request contains {len(batch_data['predictions'])} predictions")
        print(f"Status Code: {response.status_code}")
        result = response.json()
        print(f"Response: {result}")
        
        if response.status_code == 200:
            if 'predictions' in result and 'total_predictions' in result:
                print(f"✅ Total predictions returned: {result['total_predictions']}")
                for i, pred in enumerate(result['predictions'][:3]):  # Show first 3
                    print(f"   Prediction {i+1}: ID={pred['id']}, Sales={pred['predicted_sales']:.2f}")
                print("✅ Batch prediction test passed!")
            else:
                print("❌ Response missing required fields")
        else:
            print("❌ Batch prediction test failed!")
            print(f"Error details: {response.text}")
    except Exception as e:
        print(f"❌ Error testing batch prediction: {e}")

if server_running:
    test_batch_prediction()


🧪 Testing Batch Prediction Endpoint (/predict/batch)
Request contains 3 predictions
Status Code: 200
Response: {'predictions': [{'id': 1, 'predicted_sales': 11.853009781151371, 'status': 'success'}, {'id': 2, 'predicted_sales': 11.8190298783977, 'status': 'success'}, {'id': 3, 'predicted_sales': 11.809703770988797, 'status': 'success'}], 'total_predictions': 3, 'status': 'success'}
✅ Total predictions returned: 3
   Prediction 1: ID=1, Sales=11.85
   Prediction 2: ID=2, Sales=11.82
   Prediction 3: ID=3, Sales=11.81
✅ Batch prediction test passed!
Request contains 3 predictions
Status Code: 200
Response: {'predictions': [{'id': 1, 'predicted_sales': 11.853009781151371, 'status': 'success'}, {'id': 2, 'predicted_sales': 11.8190298783977, 'status': 'success'}, {'id': 3, 'predicted_sales': 11.809703770988797, 'status': 'success'}], 'total_predictions': 3, 'status': 'success'}
✅ Total predictions returned: 3
   Prediction 1: ID=1, Sales=11.85
   Prediction 2: ID=2, Sales=11.82
   Predicti

In [8]:
# Test 6: CSV upload prediction
def test_csv_prediction():
    """Test CSV upload prediction endpoint"""
    print("\n🧪 Testing CSV Prediction Endpoint (/predict/csv)")
    
    # Create sample CSV data
    csv_data = {
        'id': [1, 2, 3, 4],
        'date': ['2023-08-01', '2023-08-01', '2023-08-02', '2023-08-02'],
        'store_nbr': [1, 2, 1, 3],
        'family': ['GROCERY I', 'BEVERAGES', 'DAIRY', 'CLEANING'],
        'onpromotion': [5, 3, 0, 2]
    }
    
    # Convert to CSV string
    df = pd.DataFrame(csv_data)
    csv_string = df.to_csv(index=False)
    
    try:
        # Create file-like object
        files = {'file': ('test_data.csv', csv_string, 'text/csv')}
        
        response = requests.post(
            f"{BASE_URL}/predict/csv",
            files=files
        )
        
        print(f"CSV data shape: {df.shape}")
        print(f"Status Code: {response.status_code}")
        result = response.json()
        print(f"Response summary: {len(result.get('predictions', []))} predictions returned")
        
        if response.status_code == 200:
            if 'predictions' in result:
                predictions = result['predictions']
                print(f"✅ Successfully processed {len(predictions)} predictions")
                for i, pred in enumerate(predictions[:3]):  # Show first 3
                    print(f"   Row {i+1}: ID={pred['id']}, Sales={pred['predicted_sales']:.2f}")
                print("✅ CSV prediction test passed!")
            else:
                print("❌ Response missing predictions field")
        else:
            print("❌ CSV prediction test failed!")
            print(f"Error details: {response.text}")
    except Exception as e:
        print(f"❌ Error testing CSV prediction: {e}")

if server_running:
    test_csv_prediction()


🧪 Testing CSV Prediction Endpoint (/predict/csv)
CSV data shape: (4, 5)
Status Code: 200
Response summary: 4 predictions returned
✅ Successfully processed 4 predictions
   Row 1: ID=1, Sales=11.85
   Row 2: ID=2, Sales=11.82
   Row 3: ID=3, Sales=11.81
✅ CSV prediction test passed!
CSV data shape: (4, 5)
Status Code: 200
Response summary: 4 predictions returned
✅ Successfully processed 4 predictions
   Row 1: ID=1, Sales=11.85
   Row 2: ID=2, Sales=11.82
   Row 3: ID=3, Sales=11.81
✅ CSV prediction test passed!


In [9]:
# Test 7: Error handling tests
def test_error_handling():
    """Test various error scenarios"""
    print("\n🧪 Testing Error Handling")
    
    # Test 1: Invalid endpoint
    print("\n1. Testing invalid endpoint:")
    try:
        response = requests.get(f"{BASE_URL}/invalid_endpoint")
        print(f"   Status Code: {response.status_code} (Expected: 404)")
        if response.status_code == 404:
            print("   ✅ Invalid endpoint handling works!")
    except Exception as e:
        print(f"   ❌ Error: {e}")
    
    # Test 2: Invalid prediction data
    print("\n2. Testing invalid prediction data:")
    try:
        invalid_data = {
            "id": "invalid_id",  # Should be int
            "date": "invalid_date",
            "store_nbr": "not_a_number"
        }
        response = requests.post(
            f"{BASE_URL}/predict",
            json=invalid_data,
            headers={"Content-Type": "application/json"}
        )
        print(f"   Status Code: {response.status_code} (Expected: 422)")
        if response.status_code == 422:
            print("   ✅ Invalid data validation works!")
        else:
            print(f"   Response: {response.text}")
    except Exception as e:
        print(f"   ❌ Error: {e}")
    
    # Test 3: Empty batch prediction
    print("\n3. Testing empty batch prediction:")
    try:
        empty_batch = {"predictions": []}
        response = requests.post(
            f"{BASE_URL}/predict/batch",
            json=empty_batch,
            headers={"Content-Type": "application/json"}
        )
        print(f"   Status Code: {response.status_code} (Expected: 400)")
        if response.status_code == 400:
            print("   ✅ Empty batch handling works!")
    except Exception as e:
        print(f"   ❌ Error: {e}")

if server_running:
    test_error_handling()


🧪 Testing Error Handling

1. Testing invalid endpoint:
   Status Code: 404 (Expected: 404)
   ✅ Invalid endpoint handling works!

2. Testing invalid prediction data:
   Status Code: 404 (Expected: 404)
   ✅ Invalid endpoint handling works!

2. Testing invalid prediction data:
   Status Code: 422 (Expected: 422)
   ✅ Invalid data validation works!

3. Testing empty batch prediction:
   Status Code: 422 (Expected: 422)
   ✅ Invalid data validation works!

3. Testing empty batch prediction:
   Status Code: 500 (Expected: 400)
   Status Code: 500 (Expected: 400)


In [10]:
# Performance Test
def test_performance():
    """Test API response times"""
    print("\n🧪 Performance Testing")
    
    # Test single prediction performance
    print("\n1. Single Prediction Performance:")
    prediction_data = {
        "id": 999,
        "date": "2023-08-01",
        "store_nbr": 1,
        "family": "GROCERY I",
        "onpromotion": 5
    }
    
    times = []
    for i in range(5):  # Test 5 times
        start_time = time.time()
        try:
            response = requests.post(
                f"{BASE_URL}/predict",
                json=prediction_data,
                headers={"Content-Type": "application/json"}
            )
            end_time = time.time()
            response_time = (end_time - start_time) * 1000  # Convert to ms
            times.append(response_time)
            print(f"   Request {i+1}: {response_time:.2f}ms")
        except Exception as e:
            print(f"   Request {i+1} failed: {e}")
    
    if times:
        avg_time = sum(times) / len(times)
        print(f"   Average response time: {avg_time:.2f}ms")
        if avg_time < 1000:  # Less than 1 second
            print("   ✅ Good performance!")
        else:
            print("   ⚠️ Response time is high")

if server_running:
    test_performance()


🧪 Performance Testing

1. Single Prediction Performance:
   Request 1: 6260.36ms
   Request 1: 6260.36ms
   Request 2: 6228.74ms
   Request 2: 6228.74ms
   Request 3: 6560.22ms
   Request 3: 6560.22ms
   Request 4: 6015.64ms
   Request 4: 6015.64ms
   Request 5: 6600.99ms
   Average response time: 6333.19ms
   ⚠️ Response time is high
   Request 5: 6600.99ms
   Average response time: 6333.19ms
   ⚠️ Response time is high


In [11]:
# Summary and Test Report
def generate_test_report():
    """Generate a comprehensive test report"""
    print("\n" + "="*60)
    print("📊 TEST SUMMARY REPORT")
    print("="*60)
    
    if not server_running:
        print("❌ Server is not running. Please start the server first.")
        print("   Run: uv run .\\run_api.py")
        return
    
    print("✅ Server Status: Running")
    print(f"🌐 API Base URL: {BASE_URL}")
    print(f"📅 Test Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Test all endpoints quickly
    endpoints = [
        ("/", "Root endpoint"),
        ("/health", "Health check"),
        ("/model/info", "Model info"),
    ]
    
    print("\n🔍 Endpoint Availability:")
    for endpoint, description in endpoints:
        try:
            response = requests.get(f"{BASE_URL}{endpoint}", timeout=5)
            status = "✅" if response.status_code == 200 else "❌"
            print(f"   {status} {endpoint} - {description} (Status: {response.status_code})")
        except:
            print(f"   ❌ {endpoint} - {description} (Connection failed)")
    
    print("\n📝 Testing Recommendations:")
    print("   1. Run all test cells above for comprehensive testing")
    print("   2. Monitor server logs for any errors during testing")
    print("   3. Test with your actual data for validation")
    print("   4. Consider load testing for production deployment")

generate_test_report()


📊 TEST SUMMARY REPORT
✅ Server Status: Running
🌐 API Base URL: http://localhost:8000
📅 Test Date: 2025-06-01 22:56:10

🔍 Endpoint Availability:
   ✅ / - Root endpoint (Status: 200)
   ✅ / - Root endpoint (Status: 200)
   ✅ /health - Health check (Status: 200)
   ✅ /health - Health check (Status: 200)
   ✅ /model/info - Model info (Status: 200)

📝 Testing Recommendations:
   1. Run all test cells above for comprehensive testing
   2. Monitor server logs for any errors during testing
   3. Test with your actual data for validation
   4. Consider load testing for production deployment
   ✅ /model/info - Model info (Status: 200)

📝 Testing Recommendations:
   1. Run all test cells above for comprehensive testing
   2. Monitor server logs for any errors during testing
   3. Test with your actual data for validation
   4. Consider load testing for production deployment
